In [ ]:
import chess
import chess.engine
import numpy
from IPython.display import clear_output
import math
import time
from collections import defaultdict
import numpy as np

In [ ]:
from utils import *

In [ ]:
model_white = build_model(32, 4)
model_black = build_model(32, 4)

model_white.load_weights('model_white.h5')
model_black.load_weights('model_black.h5')

In [ ]:
model_white_64_5 = build_model(64, 5)
model_black_64_5 = build_model(64, 5)

model_white_64_5.load_weights('model_white_64_5.h5')
model_black_64_5.load_weights('model_black_64_5.h5')

In [ ]:
from abc import ABC, abstractmethod
 
class Player(ABC):
    @abstractmethod
    def choose_move(self,board):
        pass
    
class Human(ABC):
    def choose_move(self,board):
        legal_moves = self.print_moves(board)
        user_move = None
        while not user_move in legal_moves:
            user_move = input('choose move:')
        return chess.Move.from_uci(user_move)
    
    def print_moves(self,board):
        print("legal moves:")
        moves = []
        for move in board.legal_moves:
            moves.append(str(move))
            print(move,end=', ')
        return set(moves)

class AiPlayer(ABC):
    @abstractmethod
    def score(self,board):
        # how good is a position based on perspective of entity which has the turn?
        pass

In [ ]:
class ChessAi(Player,AiPlayer):
    def __init__(self,white_model,black_model,depth=2):
        self.white_model=white_model
        self.black_model=black_model
        self.depth=depth
        
    def choose_move(self,board):
        print('ai is thinking....')
        
        move = self.choose_move_minmax(board,self.depth)[0]
        return move
    
    def score(self,board):
        if board.turn:
            return 100 * eval_board(board,self.white_model)
        else:
            return 100 * (1 - eval_board(board,self.white_model))
    
    def choose_move_minmax(self,board,depth,maximize=None):
        
        if maximize is None:
            maximize = (depth%2 == 0)
            
        # minimize opponents score
        if maximize:
            score = -math.inf
        else:
            score = math.inf
        
        for move in board.legal_moves:
            board.push(move)

            if depth == 1:
                potential_score = self.score(board)
            else:
                potential_score = self.choose_move_minmax(board,depth-1,not maximize)[1]
            
            if (maximize and potential_score>score) or (not maximize and potential_score<score):
                score = potential_score
                selected_move = move
                
            board.pop()
            
        return selected_move,score

In [ ]:
chess_ai = ChessAi(model_white,model_black,2)
chess_ai_deep = ChessAi(model_white_64_5,model_black_64_5)
human_player = Human()

In [ ]:
player_black = human_player
player_white = chess_ai

In [ ]:
board = chess.Board()

In [ ]:
t_start,t_end = None,None

while not board.is_game_over():
    clear_output()
    display(board)
    
    if t_end:
        print(f'took {t_end-t_start}s to choose last move')
    
    if board.turn:
        current_player = player_white
    else:
        current_player = player_black
    
    t_start = time.time()
    move = current_player.choose_move(board)
    t_end = time.time()
    print(f'ai took {t_end-t_start}s to choose a move')
    
    board.push(move)
    
board.outcome()

In [ ]:
board

In [ ]:
import time
board_test = chess.Board()
for move in board.move_stack:
    board_test.push(move)
    clear_output()
    display(board_test)
    time.sleep(2)

In [ ]:
board.legal_moves

In [ ]:
def default_evaluate(boards):
    # placeholder evaluation function
    return list(range(len(boards)))

def produce_pruner_by_percentage(prune_percentage):
    # placeholder pruner function
    def prune(boards,scores,depth,turn):
        boards_and_scores = list(zip(scores,boards))

        # min extractor
        extractor = lambda a: a[0]
        if (turn and depth%2) or (not turn and not depth%2):
            # max extractor
            extractor = lambda a: -a[0]

        boards_and_scores.sort(key = extractor)
        boards_and_scores = boards_and_scores[:math.ceil((1-prune_percentage)*len(boards_and_scores))]
        return [board for score,board in boards_and_scores]
    return prune

default_prune = produce_pruner_by_percentage(0.5)

def produce_pruner_by_top_k(top_k):
    # placeholder pruner function
    def prune(boards,scores,depth,turn):
        boards_and_scores = list(zip(scores,boards))

        # min extractor
        extractor = lambda a: a[0]
        if (turn and depth%2) or (not turn and not depth%2):
            # max extractor
            extractor = lambda a: -a[0]

        boards_and_scores.sort(key = extractor)
        boards_and_scores = boards_and_scores[:top_k]
        return [board for score,board in boards_and_scores]
    return prune

In [ ]:
result = defaultdict(list)
def move_tree_dfs(result,board,max_depth,current_depth=1,source_move=None):
    if current_depth > max_depth: return
    for move in board.legal_moves:
        board.push(move)
        candidate_board = board.copy()
        board.pop()
        result[current_depth].append((candidate_board,source_move))
        
        if source_move is None:
            get_move_tree(result,candidate_board,max_depth,current_depth+1,move)
        else:
            get_move_tree(result,candidate_board,max_depth,current_depth+1,source_move)

In [ ]:
def move_tree_bfs(
    result,boards,max_depth,source_board_turn,current_depth=1,source_move=None,
    pruner=default_prune,evaluator=default_evaluate
):
    if type(boards) != list:
        print(boards)
        boards = [boards]
    if current_depth > max_depth: return
    for board in boards:
        for move in board.legal_moves:
            board.push(move)
            candidate_board = board.copy()
            board.pop()
            candidate_source_move = source_move
            if candidate_source_move is None:
                candidate_source_move = move
            result[current_depth].append((candidate_board,candidate_source_move))
    
    # evaluate boards
    scores = evaluator(result[current_depth])
    
    # prune boards
    print(f'original length at depth {current_depth}: {len(result[current_depth])}')
    pruned_board_set = pruner(result[current_depth],scores,current_depth,source_board_turn)
    result[current_depth] = pruned_board_set
    print(f'pruned length at depth {current_depth}: {len(result[current_depth])}')
    pruned_boards = [board for board,move in pruned_board_set]
    
    move_tree_bfs(
        result,pruned_boards,max_depth,source_board_turn,
        current_depth+1,source_move,
        pruner,evaluator
    )
    

In [ ]:
board = chess.Board()

In [ ]:
depth = 1
result = defaultdict(list)
move_tree_bfs(result,board,depth,board.turn)

In [ ]:
result[depth][0]

In [ ]:
board

In [ ]:
result[5][0]

In [ ]:
result = defaultdict(list)
move_tree_bfs(result,board,5,board.turn,pruner=produce_pruner_by_top_k(10000))

In [ ]:
str(board)

In [ ]:
board = chess.Board()

In [ ]:
widths = []
for k,v in result.items():
    print(k,len(v))
    widths.append(len(v))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.log(widths))
plt.title('logarithmic scaling of depth with tree width')
plt.xlabel('depths')
plt.ylabel('Tree Width')